In [1]:
import pandas as pd
import numpy as np
# Ensure 'pyxlsb' is installed
try:
    import pyxlsb
except ImportError:
    !pip install pyxlsb
pd.set_option('display.max_rows', 400)    

In [2]:
# Path to the Excel file
file_path = 'SRF_GSTR2_Consolidated_Excel_R_070524_Apr-24.xlsb'
# Load the Excel file
excel_file = pd.ExcelFile(file_path)
# Get the sheet names
sheets = excel_file.sheet_names

In [3]:
def has_special_characters(col):
    for char in col:
        if not (char.isalnum() or char == '_'):
            return True
    return False

In [13]:
for sheet in sheets:
    df = pd.read_excel(file_path, sheet_name=sheet,nrows=0)
    print(sheet,' ',df.shape)

B2B   (0, 96)
B2B UR   (0, 95)
CASH   (0, 110)
CASH UR   (0, 95)
CDN   (0, 95)
CDN UR   (0, 97)
IMPG   (0, 95)
IMPS   (0, 96)
B2B IRISO   (0, 76)
XDO_METADATA   (0, 1)


In [17]:
# Define your list of columns
# ie use use what columns that is necessery
l1 = [
    "INVOICE_DT", "INVOICE_NO", "SUPPLIER_NAME", "SUPPLIER_GSTIN", "SUPPLIER_STATE", "ITEM_TAXABLE_VALUE",
    "CGST_RATE", "CGST_AMOUNT", "SGST_RATE", "SGST_AMOUNT", "IGST_RATE", "IGST_AMOUNT", "CESS_RATE", "CESS_AMOUNT",
    "ELIGIBILITY_OF_ITC", "CGST_ITC_CLAIM_AMOUNT", "SGST_ITC_CLAIM_AMOUNT", "IGST_ITC_CLAIM_AMOUNT",
    "CESS_ITC_CLAIM_AMOUNT", "DBT_CRDT_NOTE_DT", "DBT_CRDT_NOTE_NO", "TRANS_TYPE_CODE",
    "Is this a Bill of Supply", "IS_REVERSE_CHARGE_APPLICABLE", "TRANS_TYPE_CODE",
    "BILL_OF_ENTRY_PORT_CODE", "BILL_OF_ENTRY_NUMBER", "BILL_OF_ENTRY_DATE", "FP", "SELF_GSTIN"
]

In [18]:
# Loop through each sheet
for sheet in sheets:
    # Read only the header (first row) to get column names
    df1 = pd.read_excel(file_path, sheet_name=sheet, nrows=0)
    # Initialize an empty list for missing columns
    missing_columns = []
    
    # Check each column in l1
    for col in l1:
        # If the column is not in df1.columns, add it to missing_columns
        if col not in df1.columns:
            missing_columns.append(col)
    
    # Print the missing columns for each sheet
    if missing_columns:
        print(f"Missing columns in sheet '{sheet}': {missing_columns}")
    else:
        print(f"No missing columns in sheet '{sheet}'.")


Missing columns in sheet 'B2B': ['Is this a Bill of Supply']
Missing columns in sheet 'B2B UR': ['Is this a Bill of Supply']
Missing columns in sheet 'CASH': ['Is this a Bill of Supply']
Missing columns in sheet 'CASH UR': ['Is this a Bill of Supply']
Missing columns in sheet 'CDN': ['Is this a Bill of Supply']
Missing columns in sheet 'CDN UR': ['Is this a Bill of Supply']
Missing columns in sheet 'IMPG': ['Is this a Bill of Supply']
Missing columns in sheet 'IMPS': ['Is this a Bill of Supply']
Missing columns in sheet 'B2B IRISO': ['Is this a Bill of Supply']
Missing columns in sheet 'XDO_METADATA': ['INVOICE_DT', 'INVOICE_NO', 'SUPPLIER_NAME', 'SUPPLIER_GSTIN', 'SUPPLIER_STATE', 'ITEM_TAXABLE_VALUE', 'CGST_RATE', 'CGST_AMOUNT', 'SGST_RATE', 'SGST_AMOUNT', 'IGST_RATE', 'IGST_AMOUNT', 'CESS_RATE', 'CESS_AMOUNT', 'ELIGIBILITY_OF_ITC', 'CGST_ITC_CLAIM_AMOUNT', 'SGST_ITC_CLAIM_AMOUNT', 'IGST_ITC_CLAIM_AMOUNT', 'CESS_ITC_CLAIM_AMOUNT', 'DBT_CRDT_NOTE_DT', 'DBT_CRDT_NOTE_NO', 'TRANS_TYPE_C

In [4]:
# Initialize an empty list to hold the rows
table_data = []
all_col = []
problem_cols = []
# Loop through each sheet
for sheet in sheets:
    # Read the data from the current sheet
    df = pd.read_excel(file_path, sheet_name=sheet,nrows=0)
    # replace spaces with underscore for sheet name ofr the csv
    sheet = sheet.replace(' ', '_')
    # Loop through each column name in the current sheet
    for col in df.columns:
        all_col.append(col)

        # Check if the column name contains 'unnamed' (case insensitive) or has special characters
        if 'unnamed' in col.lower() or has_special_characters(col):
            # Append the problematic column to the list (assuming sheet is available)
            problem_cols.append(col)

        
        # Create a tuple with table name, column name, and type
        row = (sheet, col, "VARCHAR")
        # Append the tuple to the table_data list
        table_data.append(row)
# Create a DataFrame from the list of tuples
table_schema1 = pd.DataFrame(table_data, columns=["TABLE", "COLUMN", "TYPE"])    

In [5]:
table_schema1

,TABLE,COLUMN,TYPE
0,B2B,TRANS_TYPE_CODE,VARCHAR
1,B2B,BSV,VARCHAR
2,B2B,FP,VARCHAR
3,B2B,ACK_SR_NO,VARCHAR
4,B2B,SELF_GSTIN,VARCHAR
...,...,...,...
851,B2B_IRISO,DIFF_CESS_AMNT,VARCHAR
852,B2B_IRISO,OR_DBT_CRDT_NOTE_NO,VARCHAR
853,B2B_IRISO,OR_DBT_CRDT_NOTE_DT,VARCHAR
854,B2B_IRISO,ACCOUNTED_FLAG,VARCHAR


In [6]:
for col in all_col:
    # Check if the column name contains 'unnamed' (case insensitive)
    if 'unnamed' in col.lower():
        # Print the column name
        print(col)

Unnamed: 86
Unnamed: 84


In [7]:
table_schema1.to_csv('table_schema.csv', index=False)

In [8]:
table_schema1.head()

,TABLE,COLUMN,TYPE
0,B2B,TRANS_TYPE_CODE,VARCHAR
1,B2B,BSV,VARCHAR
2,B2B,FP,VARCHAR
3,B2B,ACK_SR_NO,VARCHAR
4,B2B,SELF_GSTIN,VARCHAR


In [9]:
table_schema1['TABLE'].unique()

array(['B2B', 'B2B_UR', 'CASH', 'CASH_UR', 'CDN', 'CDN_UR', 'IMPG',
       'IMPS', 'B2B_IRISO', 'XDO_METADATA'], dtype=object)

In [10]:
table_schema1.shape

(856, 3)

In [11]:
problem_cols

['Unnamed: 86', 'Unnamed: 84']

In [12]:
for col in table_schema1['COLUMN']:
    if ' ' in col:
        print(col)

Unnamed: 86
Unnamed: 84
